In [4]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import os
import base64
import cv2
from anthropic import AsyncAnthropic
from tqdm import tqdm

In [5]:
LINK_PREFIX = "https://docs.manim.community/en/stable/"
MEDIA_OUTPUT_DIR = "downloaded_media"
SUMMARY_OUTPUT_DIR = "summary"
INPUT_DIR = "page_content"

client = AsyncAnthropic()

In [6]:
def link_to_file_name(link):
    return link[39:].replace('/', '.')

In [7]:
def extract_text_and_media(path):
    with open(path, "rb") as f:
        data = f.read()
    soup = BeautifulSoup(data, "html.parser")
    media_tags = soup.find_all(["img", "video"])
    marker = "<<<MEDIA>>>"
    media_list = []

    # Get images and replace all of them with marker
    for tag in media_tags:
        if tag.name == "img":
            src = tag.get("src") or tag.get("data‑src")
            media_list.append(("image", src))
        else:
            src = tag.get("src")
            if not src and tag.find("source"):
                src = tag.find("source").get("src")
            media_list.append(("video", src))
        tag.replace_with(marker)

    # Get all the texts and split based on marker
    full_text = soup.get_text()
    parts = [piece.strip() for piece in full_text.split(marker)]

    # Interleave
    result = []
    for i, media in enumerate(media_list):
        if parts[i]:
            result.append(("text", parts[i]))
        result.append(media)
    if len(parts) > len(media_list) and parts[-1]:
        result.append(("text", parts[-1]))

    return result

In [8]:
def save_images_and_videos(url, content):
    session = requests.Session()

    for t, info in content:
        if t == "image" or t == "video":
            full_url = urljoin(url, info)
            response = session.get(full_url)

            if response.status_code == 200:
                with open(f"{MEDIA_OUTPUT_DIR}/{link_to_file_name(full_url)}", "wb") as f:
                    f.write(response.content)
                print(f"[saved] {link_to_file_name(full_url)}")
            else:
                print(f"[failed] {full_url} (status code {response.status_code})")

In [10]:
def construct_message(url, content):
    message_content = []

    for t, info in content:
        if t == "image":
            full_url = urljoin(url, info)
            path = os.path.join(MEDIA_OUTPUT_DIR, link_to_file_name(full_url))

            # No svg
            if path[-3:] == "svg":
                continue

            with open(path, "rb") as f:
                image_data = f.read()

            message_content.append({
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/png",
                    "data": base64.standard_b64encode(image_data).decode("utf-8"),
                }
            })

        elif t == "video":
            # Extract middle frame
            full_url = urljoin(url, info)
            path = os.path.join(MEDIA_OUTPUT_DIR, link_to_file_name(full_url))
            cap = cv2.VideoCapture(path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            middle_frame_idx = frame_count // 2

            cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_idx)
            _, frame = cap.read()
            cap.release()
            _, buffer = cv2.imencode(".png", frame)
            image_data = buffer.tobytes()

            message_content.append({
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/png",
                    "data": base64.standard_b64encode(image_data).decode("utf-8"),
                }
            })
        else:
            message_content.append({ "type": "text", "text": info })
    
    return { "role": "user", "content": message_content }

In [12]:
def run_save():
    for file_name in os.listdir(INPUT_DIR):
        path = os.path.join(INPUT_DIR, file_name)
        
        with open(path, "rb") as f:
            url = f.readline().decode('utf-8').strip()
        
        content = extract_text_and_media(path)
        save_images_and_videos(url, content)

# run_save()

In [25]:
# Construct all summaries
results = []

for file_name in tqdm(os.listdir(INPUT_DIR)):
    path = os.path.join(INPUT_DIR, file_name)
    
    with open(path, "rb") as f:
        url = f.readline().decode('utf-8').strip()
    
    content = extract_text_and_media(path)
    user_message = [construct_message(url, content)]
    
    query = client.messages.create(
        model="claude-3-5-haiku-20241022",
        temperature=1,
        max_tokens=8192,
        system="You are an expert technical writer. You will be given the content of some documentation for the Manim library. Please summarize the information you are given but do so such that someone with no experience can still get the same amount of information from the summary (feel free to include the examples) but also try to keep it as short as possible to minimize token use (pretty important). Also, if you believe the information is not relevant or important for an average user to code in Manim, you can simply say 'Not relevant' -- keep your standards high, for example something like the changelog or how to contribute isn't important. Just to reiterate, unimportant things like the changelog should not be summarized and you should just say 'Not relevant' -- really make sure that only the actually absolute necessary things in the documentation are summarized. Only include your reponse/summary and absolutely nothing more, don't have any unnecessary comments (since minimizing response length is really important). To reiterate again, make the response as short as possible!",
        messages=user_message
    )
    results.append((file_name, query))

C:\Users\alexl\AppData\Local\Temp\ipykernel_4140\901300213.py:2: RuntimeWarning: coroutine 'AsyncMessages.create' was never awaited
  results = []
100%|██████████| 539/539 [01:07<00:00,  7.98it/s]


In [26]:
for file_name, query in tqdm(results):
    query = await query
    info = query.content[0].text

    with open(os.path.join(SUMMARY_OUTPUT_DIR, file_name), "w", encoding="utf-8") as f:
        f.write(info)

100%|██████████| 539/539 [1:28:40<00:00,  9.87s/it]
